In [51]:
%matplotlib inline

In [52]:
import matplotlib.pyplot as plt
import pandas as pd
import requests as rq

In [69]:
r = rq.get('https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2010-01-01&endtime=2020-12-31&limit=1000&minmagnitude=7')
raw_data = r.json()

In [70]:
df = pd.json_normalize(raw_data["features"])[["id", "properties.time", "properties.mag", 
                                              "properties.status", "geometry.coordinates"]]

df = pd.concat([df.drop(['geometry.coordinates'], axis=1), df['geometry.coordinates'].apply(pd.Series)], axis=1)
df = df.rename(columns={0: "lat", 1: "lon", 2: "depth"}, errors="raise")

df

,id,properties.time,properties.mag,properties.status,lat,lon,depth
0,us7000asvb,1595398364719,7.8,reviewed,-158.5543,55.0683,28.0
1,us7000aq3e,1594954222158,7.0,reviewed,147.7755,-7.8392,73.0
2,us6000ah9t,1592926144350,7.4,reviewed,-96.0077,15.8861,20.0
3,us6000ae4n,1592484593844,7.4,reviewed,-177.8571,-33.2927,10.0
4,us70008fi4,1585104561160,7.5,reviewed,157.6955,48.9638,57.8
...,...,...,...,...,...,...,...
162,usp000h7sq,1267257683010,7.4,reviewed,-75.0480,-37.7730,35.0
163,official20100227063411530_30,1267252451530,8.8,reviewed,-72.8980,-36.1220,22.9
164,usp000h7qu,1267216286970,7.0,reviewed,128.4250,25.9300,25.0
165,usp000h60h,1263333190060,7.0,reviewed,-72.5710,18.4430,13.0
